In [17]:
import numpy as np
from IPython.display import clear_output
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.datasets import imdb
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder

from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.models import load_model

import import_notebook
from readWrite import ReadWrite
from vectorToDoc import VectorToDoc
from docToVector import DocToVector
from dictionary import Dictionary

In [2]:
tweetFile = "combineVectors.csv"
tweetClassFile = "combineVectorsResult.txt"

readWrite = ReadWrite()
xdata = readWrite.readFile(tweetFile)
ydata = readWrite.readFileClassifier(tweetClassFile)

x_train = xdata[:int(len(xdata) * 0.8) + 1]
y_train = ydata[:int(len(ydata) * 0.8) + 1]
x_test = xdata[int(len(xdata) * 0.8):-1]
y_test = ydata[int(len(ydata) * 0.8):-1]

print("x_train: {}, shape: {}".format(len(x_train), x_train.shape))
print("y_train: {}, shape: {}".format(len(y_train), y_train.shape))
print("x_test: {}, shape: {}".format(len(x_test), x_test.shape))
print("y_test: {}, shape: {}".format(len(y_test), y_test.shape))

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(ydata)
encoded = encoder.transform(ydata)
# convert integers to dummy variables (i.e. one hot encoded)
encoding = np_utils.to_categorical(encoded)

encoding_train = encoding[:int(len(xdata) * 0.8) + 1]
encoding_test = encoding[int(len(ydata) * 0.8):-1]
print("encoding_train shape: {}".format(encoding_train.shape))
print("encoding_test shape: {}".format(encoding_test.shape))

line size more than 20, trim to: ['2356', ' 1257', ' 1689', ' 1391', ' 2330', ' 791', ' 80', ' 1060', ' 2002', ' 1884', ' 1060', ' 300', ' 2330', ' 1135', ' 1799', ' 2073', ' 1165', ' 2253', ' 185', ' 1207']
line size more than 20, trim to: ['2356', ' 1257', ' 1239', ' 500', ' 851', ' 1568', ' 1932', ' 851', ' 1239', ' 2153', ' 138', ' 1239', ' 455', ' 1008', ' 1044', ' 21', ' 1243', ' 1302', ' 851', ' 1503']
Read complete, total records: 1814
Read complete, total records: 1814
x_train: 1452, shape: (1452, 20)
y_train: 1452, shape: (1452,)
x_test: 362, shape: (362, 20)
y_test: 362, shape: (362,)
encoding_train shape: (1452, 3)
encoding_test shape: (362, 3)


In [52]:
vocabSize = 3000
outputDim = 64
recordsSize = 20

model = Sequential()
model.add(Embedding(vocabSize, outputDim, input_length=recordsSize))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.8))
model.add(Dense(3, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 20, 64)            192000    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 64)                24832     
_________________________________________________________________
dropout_4 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 195       
Total params: 217,027
Trainable params: 217,027
Non-trainable params: 0
_________________________________________________________________


In [56]:
batch_size = 32
epochs = 30

# while(True):        
#     model.fit(x_train, encoding_train, validation_data=(x_test, encoding_test), epochs=epochs, batch_size=outputDim, verbose=2)
#     scores = model.evaluate(x_test, encoding_test, verbose=0)    
#     print("Accuracy: %.2f%%" % (scores[1]*100))
#     if(scores[1] > 0.85):
#         break
#     else:
#         clear_output()

model.fit(x_train, encoding_train, epochs=epochs, 
          batch_size=batch_size, validation_data=[x_test, encoding_test])
scores = model.evaluate(x_test, encoding_test, verbose=0)    
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 1452 samples, validate on 362 samples
Epoch 1/30
1452/1452 [==============================] - 4s - loss: 0.0094 - acc: 0.9975 - val_loss: 1.2731 - val_acc: 0.8435
Epoch 2/30
1452/1452 [==============================] - 4s - loss: 0.0106 - acc: 0.9982 - val_loss: 1.4445 - val_acc: 0.8490
Epoch 3/30
1452/1452 [==============================] - 4s - loss: 0.0123 - acc: 0.9961 - val_loss: 1.3557 - val_acc: 0.8481
Epoch 4/30
1452/1452 [==============================] - 4s - loss: 0.0112 - acc: 0.9975 - val_loss: 1.5297 - val_acc: 0.8499
Epoch 5/30
1452/1452 [==============================] - 4s - loss: 0.0303 - acc: 0.9920 - val_loss: 1.2996 - val_acc: 0.8508
Epoch 6/30
1452/1452 [==============================] - 4s - loss: 0.0213 - acc: 0.9963 - val_loss: 1.0690 - val_acc: 0.8306
Epoch 7/30
1452/1452 [==============================] - 4s - loss: 0.0124 - acc: 0.9970 - val_loss: 1.3006 - val_acc: 0.8490
Epoch 8/30
1452/1452 [==============================] - 4s - loss: 0.0116 - ac

In [57]:
'''
Saving model or load model
'''
model.save('model/bidir_model_Laptop.h5')
# model = load_model('model/bidir_model_Laptop.h5')

scores = model.evaluate(x_test, encoding_test, verbose=0)    
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 84.35%


In [58]:
'''
Classify sentence
0-unknown, 1-update, 2-delay
'''
def classify_sentence(sentence, debug=False):
    dic = Dictionary()
    vectorToDoc = VectorToDoc()
    docToVector = DocToVector()
    senArr = dic.readSentence(sentence, True)
    if(debug):
        print(senArr)

    vector = docToVector.convertVector(senArr, True)
    sen = vectorToDoc.convertDoc(vector)
    vector = np.array(vector)
    if(debug):
        print(vector)
        print(sen)

    predicted = model.predict_classes(vector)
    print(predicted[0])
    
classify_sentence("[CCL Update] please add 30mins additional travelling time from #PayaLebar to #BuonaVista due to train fault. Free Regular bus available.", True)

Dictionary count: 2307
Dictionary count: 2305
[['ccl', 'updat', 'pleas', 'add', 'min', 'addit', 'travel', 'time', 'payalebar', 'buonavista', 'due', 'train', 'fault', 'free', 'regular', 'bu', 'avail']]
Converting records to vectors...
Max length of record: 20
IsPadding enable: True
[[1795 1052  717  792 2048 1696 2001  486 1204 1341 1027 1069 2140  206
  1385 1521  767    0    0    0]]
[['ccl', 'updat', 'pleas', 'add', 'min', 'addit', 'travel', 'time', 'payalebar', 'buonavista', 'due', 'train', 'fault', 'free', 'regular', 'bu', 'avail', 'NONE', 'NONE', 'NONE']]
1/1 [==============================] - 0s
0


In [61]:
'''
0-unknown, 1-update, 2-delay
'''
dic = Dictionary()
docToVector = DocToVector()

# testArr = dic.readFile("user_tweets4.txt", True)
# vectorsArr2 = docToVector.convertVector(testArr, True)
# vectorsArr2 = np.array(vectorsArr2)
# oriRecords2 = readWrite.readOriFile('user_tweets4.txt')

vectorsArr2 = readWrite.readFile('docVectors3.csv')
oriRecords2 = readWrite.readOriFile('user_tweets3.txt')

print("vector records shape: {}".format(vectorsArr2.shape))

classResult = []
predicted = model.predict_classes(vectorsArr2)
predicted = np.reshape(predicted, (predicted.size,))

print()
print(predicted.shape)

for idx, score in enumerate(predicted):    
    if(score == 1): 
        classResult.append(oriRecords2[idx])       
        print(predicted[idx])
        print(oriRecords2[idx])        

# readWrite.writeFile('docVectors3Result.txt', predicted)

Dictionary count: 2305
Read complete, total records: 689
vector records shape: (689, 20)
672/689 [============================>.] - ETA: 0s
(689,)
1
@SMRT_Singapore Rubbish. I am on train now. The announcement and train still indicate it will terminate at Ang Mo Kio.

1
@SBSTransit_Ltd what time is the first train for the downtown line at bukit panjang on the sunday?  reply by tdy please

1
@SMRT_Singapore honestly public transport is supposed to make life more convenient but that ain't the case with y'all.

1
@SMRT_Singapore Can that someone commit hara kiri?

1
@SMRT_Singapore u bloody hell better pay me back my money for cab to town from Tpy leh legit fucked up yall

1
@SMRT_Singapore #mrt is down again and there is mess in AMK station. No officers at the platform to guide commuters.

1
@SMRT_Singapore On train and yet NOT a single announcement...well done again..

1
@SMRT_Singapore Really perfect. Just boarded to train. :(

1
@SBSTransit_Ltd #DTL  I think Chinatwn stat.. should blo